In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28679
2,Huelva,Confirmados PDIA 14 días,1085
3,Huelva,Tasa PDIA 14 días,"211,43090983494747"
4,Huelva,Confirmados PDIA 7 días,440
5,Huelva,Total Confirmados,28854
6,Huelva,Curados,24680
7,Huelva,Fallecidos,365


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28679.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7440.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1273 personas en los últimos 7 días 

Un positivo PCR cada 607 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28679.0,440.0,1085.0,513170.0,85.741567,211.430910,77.0
Huelva-Costa,16704.0,182.0,404.0,289548.0,62.856590,139.527816,40.0
Condado-Campiña,9054.0,227.0,569.0,156231.0,145.297668,364.204287,38.0
Huelva (capital),7440.0,113.0,237.0,143837.0,78.561149,164.769844,26.0
Almonte,1540.0,69.0,172.0,24507.0,281.552210,701.840291,19.0
Bollullos Par del Condado,831.0,44.0,80.0,14387.0,305.831654,556.057552,10.0
Palos de la Frontera,586.0,11.0,26.0,11742.0,93.680804,221.427355,4.0
Aljaraque,1005.0,13.0,22.0,21474.0,60.538325,102.449474,4.0
Isla Cristina,2063.0,10.0,30.0,21393.0,46.744262,140.232786,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres de San Bartolomé,11.0,1.0,6.0,376.0,265.957447,1595.744681,NaN
Lucena del Puerto,233.0,15.0,42.0,3261.0,459.981601,1287.948482,1.0
Cortelazor,7.0,2.0,3.0,299.0,668.896321,1003.344482,0.0
Cumbres Mayores,49.0,1.0,17.0,1749.0,57.175529,971.983991,0.0
Santa Ana la Real,10.0,1.0,4.0,475.0,210.526316,842.105263,NaN
Almonte,1540.0,69.0,172.0,24507.0,281.552210,701.840291,19.0
Galaroza,28.0,1.0,9.0,1382.0,72.358900,651.230101,NaN
Bollullos Par del Condado,831.0,44.0,80.0,14387.0,305.831654,556.057552,10.0
Beas,198.0,5.0,24.0,4341.0,115.180834,552.868003,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,49.0,1.0,17.0,1749.0,57.175529,971.983991,0.0,0.058824
Galaroza,28.0,1.0,9.0,1382.0,72.358900,651.230101,0.0,0.111111
Cortegana,199.0,1.0,7.0,4602.0,21.729683,152.107779,0.0,0.142857
Cumbres de San Bartolomé,11.0,1.0,6.0,376.0,265.957447,1595.744681,0.0,0.166667
Valverde del Camino,475.0,4.0,20.0,12750.0,31.372549,156.862745,0.0,0.200000
Minas de Riotinto,112.0,1.0,5.0,3812.0,26.232949,131.164743,0.0,0.200000
Beas,198.0,5.0,24.0,4341.0,115.180834,552.868003,1.0,0.208333
Sierra de Huelva-Andévalo Central,2550.0,24.0,99.0,67391.0,35.613064,146.903889,3.0,0.242424
Santa Ana la Real,10.0,1.0,4.0,475.0,210.526316,842.105263,0.0,0.250000
